In [28]:
import gc
import pandas as pd
import numpy as np

from scipy import sparse as sps
#from sparsesvd import sparsesvd
from collections import Counter 
from matplotlib import pyplot as plt


path = '../data/'

attend = pd.read_csv(path+'attend.csv')
attend.drop(['дата занятия','время начала занятия','время окончания занятия','направление 3','направление 2'], axis=1, inplace=True)

groups = pd.read_csv(path+'groups.csv')
test = pd.read_csv(path+'test.csv')
users = pd.read_csv(path + 'users.csv')
users['age']  = users['дата рождения'].apply(lambda x: int((2023-int(x.split('-')[0]))))
users['age'] = users['age'].apply(lambda x: str(x - x % 5)+'__age')
attend.head()

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,онлайн/офлайн
0,401346550,801346550,101352023,Да
1,401346550,801346550,101385462,Да
2,401346550,801346550,101421897,Да
3,401346550,801346550,101354499,Да
4,401346550,801346550,101421312,Да


In [29]:
users.head(1)


,уникальный номер,дата создание личного дела,пол,дата рождения,адрес проживания,age
0,101391104,2019-02-26 15:52:09.000,Женщина,1959-09-10,"город москва, константинова, дом 30",60__age


In [30]:
attend.head(1)

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,онлайн/офлайн
0,401346550,801346550,101352023,Да


In [31]:
def append_hours(df, users):
    
    # выделяем час заказа
   # df['online'] = df['онлайн/оффлайн'].apply(lambda x: 1 if x == 'Да' else 0)
    
    for c in ['пол','age']:
        us = users.copy()
        us.rename({'уникальный номер':'уникальный номер участника',
                   c:'уникальный номер группы'}, axis=1, inplace=True)
        us['уникальный номер занятия'] = df['уникальный номер занятия'].max() + 1 + np.arange(us.shape[0])
        us['онлайн/офлайн'] = 'Нет'

        df = pd.concat([df, us[df.columns]])
    

    

    return df

print(attend.shape)
attend = append_hours(attend, users)
print(attend.shape)

(5901274, 4)
(6005942, 4)


In [32]:
print(attend.shape)
attend = attend.drop_duplicates(subset=['уникальный номер участника','уникальный номер группы'])
print(attend.shape)
attend.head()

(6005942, 4)
(705058, 4)


,уникальный номер занятия,уникальный номер группы,уникальный номер участника,онлайн/офлайн
0,401346550,801346550,101352023,Да
1,401346550,801346550,101385462,Да
2,401346550,801346550,101421897,Да
3,401346550,801346550,101354499,Да
4,401346550,801346550,101421312,Да


In [33]:
# фильтруем на малоактивных пользователей и малопопулярные продукты
def filter_column(ds, col, min_freq, free_col='rating'):
    
    temp = ds.groupby(col, as_index=False).agg({free_col:'count'})
    filtered = temp.loc[temp[free_col] >= min_freq, col].values
    ds = ds.loc[(ds[col].isin(filtered))].copy()

    return ds


attend = filter_column(attend, col='уникальный номер группы', min_freq=10, free_col='уникальный номер занятия')
#hist_data = filter_column(hist_data, col='item_id', min_freq=20,free_col='уникальный номер занятия')

attend.rename({'уникальный номер участника':'user_id','уникальный номер группы':'item_id'}, axis=1, inplace=True)

In [34]:
user2idx = {v: k for k, v in enumerate(attend.user_id.unique())}
item2idx = {v: k for k, v in enumerate(attend.item_id.unique())}
print(len(item2idx))
idx2user = {k:v for v, k in user2idx.items()}
idx2item = {k:v for v, k in item2idx.items()}

attend['user_id'] = attend.user_id.apply(lambda x: user2idx[x])
attend['item_id'] = attend.item_id.apply(lambda x: item2idx[x])

attend.head()

14442


,уникальный номер занятия,item_id,user_id,онлайн/офлайн
0,401346550,0,0,Да
1,401346550,0,1,Да
2,401346550,0,2,Да
3,401346550,0,3,Да
4,401346550,0,4,Да


In [35]:

matrix = sps.coo_matrix(
    (np.ones(attend.shape[0]), (attend['user_id'], attend['item_id'])),
    shape=(len(user2idx), len(item2idx)),
)
matrix

<52334x14442 sparse matrix of type '<class 'numpy.float64'>'
	with 651476 stored elements in COOrdinate format>

In [9]:
%%time

# Обучаем конечную модель
# Мы взяли реализацию из RecBole
# 
def fit_ease(X, reg_weight=100):
    
 # gram matrix
    G = X.T @ X

    # add reg to diagonal
    G += reg_weight * sps.identity(G.shape[0])

    # convert to dense because inverse will be dense
    G = G.todense()

    # invert. this takes most of the time
    P = np.linalg.inv(G)
    B = P / (-np.diag(P))
    # zero out diag
    np.fill_diagonal(B, 0.)
    
    return B

w = fit_ease(matrix)

CPU times: user 5min 4s, sys: 9.61 s, total: 5min 14s
Wall time: 1min 4s


In [10]:
w.shape

(14442, 14442)

In [11]:
items_to_recommend = pd.read_csv(path+'Список групп для рекомендаций.csv', header=None)
items_to_recommend.columns = ['item_id']

norm_namings = items_to_recommend.item_id.values
norm_idxs = [item2idx[x] for x in norm_namings if x in item2idx]
print(len(norm_idxs) / len(norm_namings), len(norm_idxs))


items_to_recommend.head()

0.5637642919964819 641


,item_id
0,801362710
1,801353142
2,801362725
3,801362727
4,801362748


In [12]:
norm_namings

array([801362710, 801353142, 801362725, ..., 801372384, 801366477,
       801366823])

In [13]:
test['user_id'] = test['уникальный номер участника'].apply(lambda x: user2idx.get(x, -1))
test_data = test.merge(attend, how='left')
test_data.head()

,уникальный номер участника,уникальный номер группы,user_id,уникальный номер занятия,item_id,онлайн/офлайн
0,101346610,NaN,18734,401353854,1068,Нет
1,101346610,NaN,18734,401384586,5098,Нет
2,101346610,NaN,18734,401384808,5115,Нет
3,101346610,NaN,18734,401385151,5136,Нет
4,101346610,NaN,18734,401480748,9386,Нет


In [14]:
def get_preds(x):
    
    #basket = [item2idx[t] for t in x if t in item2idx]
    if str(x[0]) != 'nan':
        #print(x)
        basket = [int(t) for t in x]
    else:
        basket = []
        
        
    #print(basket)
    
    # Составляем вектор интеракций человека
    vector = np.zeros(len(item2idx))
    vector[basket] = 1
    
    return vector

pred = test_data.groupby(['уникальный номер участника'])['item_id'].agg([('basket', list)]).reset_index()
pred['vector'] = pred.basket.apply(lambda x: get_preds(x))
pred.head()

,уникальный номер участника,basket,vector
0,101346610,"[1068, 5098, 5115, 5136, 9386, 9387, 9470, 510...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,101346612,"[1068, 3212, 1199, 5098, 5115, 5136, 5103, 857...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,101347269,"[106, 114, 1068, 2552, 4672, 5098, 5103, 5115,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,101347321,"[1068, 5098, 5115, 5136, 9386, 9387, 7281, 728...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,101347983,"[7123, 7614, 2272, 5485, 10233, 14430, 14438]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [15]:
from tqdm import tqdm
scores = []
recs_for_user = []
print(pred.shape)

batch = []
#w = w.toarray()
for idx, row in tqdm(pred.iterrows(), total = pred.shape[0]):
    vector = row.vector
    
    batch.append(vector)
#     if len(batch) > 10000:
#         batch = np.array(batch)
#         user_scores = batch.dot(w)
#         user_scores = batch.dot(w)
#         user_scores[:,~np.array(norm_idxs)] = -np.inf
#         user_scores = np.argsort(-user_scores)[:,:200]
#         for i in range(len(user_scores)):
#             recs_for_user.append(user_scores[i].tolist())
            
#         batch = []
        
batch = np.array(batch)
user_scores = batch.dot(w)
user_scores[:,norm_idxs] += 1000

user_scores[batch==1] = -np.inf
user_scores = np.argsort(-user_scores)[:,:10]
for i in range(len(user_scores)):
    recs_for_user.append(user_scores[i].tolist())
            

(200, 3)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 12743.80it/s]


In [16]:
def get_decoded_recommendations(x):
    
    # создаем конечные декодированные рекомендации
    recs = []
    
    # смотрим, что уже лежит в корзине
    consumed = [t for t in x.basket]
    for el in x.recs[0]:
        
        # нам надо выкинуть специальные токены из рекомендаций. Это то, что связано с аггрегированными
        # статистиками в заказе. А также не положить то, что уже лежит в корзине
        cond = el not in consumed
        if cond:
            recs.append(idx2item[el])
        else:
            1/0
        if len(recs) == 10:
            break
            
    if len(recs) != 10:
        1/0
    return recs


pred['recs'] = recs_for_user
pred['preds'] = pred.apply(lambda x: get_decoded_recommendations(x), axis=1)
pred.head(10)

,уникальный номер участника,basket,vector,recs,preds
0,101346610,"[1068, 5098, 5115, 5136, 9386, 9387, 9470, 510...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[10710, 13927, 11986, 8571, 12943, 13305, 110...","[801364201, 801367538, 801370061, 801359786, 8..."
1,101346612,"[1068, 3212, 1199, 5098, 5115, 5136, 5103, 857...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[13927, 10710, 13305, 11986, 13161, 12943, 12...","[801367538, 801364201, 801363560, 801370061, 8..."
2,101347269,"[106, 114, 1068, 2552, 4672, 5098, 5103, 5115,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[12689, 11071, 12972, 13225, 14246, 13976, 13...","[801371219, 801371254, 801363286, 801372078, 8..."
3,101347321,"[1068, 5098, 5115, 5136, 9386, 9387, 7281, 728...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[10710, 13927, 11986, 8571, 11069, 12165, 129...","[801364201, 801367538, 801370061, 801359786, 8..."
4,101347983,"[7123, 7614, 2272, 5485, 10233, 14430, 14438]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[12118, 11690, 10985, 10721, 338, 10510, 1001...","[801369249, 801370592, 801368693, 801369241, 8..."
5,101348056,"[2877, 10971, 10800, 12884, 13152, 11848, 6833...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[10086, 10968, 10084, 10964, 11925, 13149, 88...","[801362750, 801364975, 801363043, 801364971, 8..."
6,101349211,"[108, 130, 732, 908, 3, 1659, 1735, 1858, 3250...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[14130, 12873, 13976, 12856, 2167, 4606, 1297...","[801368506, 801368552, 801369774, 801370296, 8..."
7,101349847,"[2375, 2783, 3071, 735, 2597, 3577, 3395, 4518...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[13160, 10380, 10393, 12893, 13729, 12992, 13...","[801371128, 801363956, 801367425, 801369799, 8..."
8,101350099,"[906, 1128, 1799, 2033, 888, 3206, 1765, 2259,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[13927, 12382, 10764, 13830, 11408, 14103, 13...","[801367538, 801371813, 801370992, 801371092, 8..."
9,101351179,"[146, 371, 3205, 3608, 582, 1652, 1723, 4447, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[10339, 11036, 12622, 14057, 9963, 2761, 1001...","[801363623, 801369287, 801368130, 801371158, 8..."


##  Проверки

In [17]:
#toppopular?
pred.explode('preds')['preds'].value_counts()[:10]

preds
801365028    23
801364172    23
801369785    20
801353958    19
801371717    18
801362957    18
801354057    17
801370323    17
801365155    17
801363677    17
Name: count, dtype: int64

In [18]:
# all from recommended?
items_recommended = pred.explode('preds')['preds'].unique()

assert set(items_recommended) < set(items_to_recommend.item_id.values.tolist())

In [19]:
#no intersections with train?

clean_recs = pred.explode('preds')
clean_recs.rename({'preds':'уникальный номер группы'}, axis=1, inplace=True)
attend2 = pd.read_csv(path+'attend.csv')
assert clean_recs.merge(attend2, on=['уникальный номер группы','уникальный номер участника']).shape[0] == 0

In [20]:
clean_recs.merge(attend2, on=['уникальный номер группы','уникальный номер участника'])

,уникальный номер участника,basket,vector,recs,уникальный номер группы,уникальный номер занятия,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия


In [21]:
pred['уникальный номер группы'] = pred.preds.apply(lambda x: ','.join(map(str, x)))
test22 = pd.read_csv(path+'test.csv')
test22.drop(['уникальный номер группы'],axis=1, inplace=True)
test22 = test22.merge(pred[['уникальный номер участника','уникальный номер группы']])
test22.to_csv('submission.csv', index=None)
test22.head(30)

,уникальный номер участника,уникальный номер группы
0,101346610,"801364201,801367538,801370061,801359786,801369..."
1,101346612,"801367538,801364201,801363560,801370061,801367..."
2,101347269,"801371219,801371254,801363286,801372078,801371..."
3,101347321,"801364201,801367538,801370061,801359786,801368..."
4,101347983,"801369249,801370592,801368693,801369241,801346..."
5,101348056,"801362750,801364975,801363043,801364971,801365..."
6,101349211,"801368506,801368552,801369774,801370296,801349..."
7,101349847,"801371128,801363956,801367425,801369799,801364..."
8,101350099,"801367538,801371813,801370992,801371092,801364..."
9,101351179,"801363623,801369287,801368130,801371158,801370..."


In [22]:
#pred.loc[pred['уникальный номер участника'].isin([101347321,101346610])].merge(users,
#                                                                               left_on=['уникальный номер участника'],
#                                                                               right_on=['уникальный номер'])

## Интерпретация

In [23]:
group2name = {row['уникальный номер']: row['направление 3'] for _, row in groups.iterrows()}


df = pred.copy()
df['visited'] = df.basket.apply(lambda x: [group2name.get(idx2item[t],None) for t in x if t in idx2item  ])
df['recommended'] = df.preds.apply(lambda x: [group2name[t] for t in x])


In [24]:
for i in range(10):
    print('-'*100)
    sample = df.sample()
    print('Бабушка посетила:')
    print(set(sample.visited.values[0]))
    print('\nБабушке рекомендовали:')
    print(set(sample.recommended.values[0]))
    print('-'*100)

----------------------------------------------------------------------------------------------------
Бабушка посетила:
{'ОНЛАЙН Иные интеллектуальные игры', 'ОНЛАЙН Мастер-класс по уходу за кожей в зрелом возрасте', 'ОНЛАЙН Тренинги личностного роста', 'ОНЛАЙН Здорово жить', 'ОНЛАЙН Дыхательная гимнастика', 'ОНЛАЙН Уход и содержание домашних животных', 'ОНЛАЙН История, культура России', 'ОНЛАЙН Цигун', 'ОНЛАЙН Основы духовной культуры', 'ОНЛАЙН История искусства', None, 'ОНЛАЙН Здоровый образ жизни (ЗОЖ)', 'ОНЛАЙН Оздоровительная гимнастика', 'ОНЛАЙН Правильное питание', 'ОНЛАЙН Психологические тренинги', 'ОНЛАЙН Суставная гимнастика', 'ОНЛАЙН История моды и прически'}

Бабушке рекомендовали:
{'ОНЛАЙН Мастер-класс по уходу за кожей в зрелом возрасте', 'ОНЛАЙН Садоводство', 'ОНЛАЙН Рисование анти-стресс', 'ОНЛАЙН История искусства', 'ОНЛАЙН Оздоровительная гимнастика', 'ОНЛАЙН Акварельная живопись', 'ОНЛАЙН Адаптивная и тонизирующая гимнастика', 'ОНЛАЙН Суставная гимнастика', 'ОНЛАЙН Зд

## Предикты на всех

In [37]:
full_data = attend.copy()

full_data = full_data.groupby(['user_id'])['item_id'].agg([('basket', list)]).reset_index()
full_data['vector'] = full_data.basket.apply(lambda x: get_preds(x))
full_data.head()

,user_id,basket,vector
0,0,"[0, 3177, 2609, 2613, 3770, 4744, 5042, 4773, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,"[0, 1796, 2333, 1604, 2777, 4437, 6243, 2388, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,"[0, 4780, 9769, 14430, 14435]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,"[0, 1130, 3173, 1665, 4727, 4951, 5647, 2609, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,"[0, 8077, 9769, 14430, 14435]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [39]:
from tqdm import tqdm
scores = []
recs_for_user = []
print(full_data.shape)

batch = []
#w = w.toarray()
for idx, row in tqdm(full_data.iterrows(), total = full_data.shape[0]):
    vector = row.vector
    
    batch.append(vector)
    if len(batch) > 1000:
        batch = np.array(batch)
        user_scores = batch.dot(w)
        user_scores[:,norm_idxs] += 1000
        user_scores[batch==1] = -np.inf
        user_scores = np.argsort(-user_scores)[:,:200]
        for i in range(len(user_scores)):
            recs_for_user.append(user_scores[i].tolist())
            
        batch = []
        
batch = np.array(batch)
user_scores = batch.dot(w)
user_scores[:,norm_idxs] += 1000

user_scores[batch==1] = -np.inf
user_scores = np.argsort(-user_scores)[:,:30]
for i in range(len(user_scores)):
    recs_for_user.append(user_scores[i].tolist())
            

(52334, 3)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52334/52334 [03:26<00:00, 253.27it/s]


In [53]:
def get_decoded_recommendations(x):
    
    # создаем конечные декодированные рекомендации
    recs = []
    
    # смотрим, что уже лежит в корзине
    consumed = [t for t in x.basket]
    for el in x.recs[0]:
        
        # нам надо выкинуть специальные токены из рекомендаций. Это то, что связано с аггрегированными
        # статистиками в заказе. А также не положить то, что уже лежит в корзине
        cond = el not in consumed
        if cond:
            recs.append(idx2item[el])
        else:
            1/0
        if len(recs) == 30:
            break
            
    if len(recs) != 30:
        1/0
    return recs


full_data['recs'] = recs_for_user
full_data['preds'] = full_data.apply(lambda x: get_decoded_recommendations(x), axis=1)
full_data.head(10)

,user_id,basket,vector,recs,preds,decoded_user_id
0,0,"[0, 3177, 2609, 2613, 3770, 4744, 5042, 4773, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[13336, 14026, 11317, 12210, 9937, 11306, 132...","[801363037, 801367596, 801367854, 801371327, 8...",101352023
1,1,"[0, 1796, 2333, 1604, 2777, 4437, 6243, 2388, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[12791, 13814, 10879, 6938, 3818, 1822, 11416...","[801368710, 801366648, 801368600, 801356150, 8...",101385462
2,2,"[0, 4780, 9769, 14430, 14435]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[13336, 14251, 3130, 12803, 13067, 12787, 119...","[801363037, 801367527, 801350278, 801370236, 8...",101421897
3,3,"[0, 1130, 3173, 1665, 4727, 4951, 5647, 2609, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[13336, 12963, 14011, 10879, 10764, 13726, 10...","[801363037, 801368178, 801370014, 801368600, 8...",101354499
4,4,"[0, 8077, 9769, 14430, 14435]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[14251, 12803, 3130, 12787, 11979, 13336, 104...","[801367527, 801370236, 801350278, 801368313, 8...",101421312
5,5,"[0, 36, 667, 1605, 2700, 3666, 4186, 4018, 516...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[13833, 12887, 6337, 7011, 9049, 8935, 11652,...","[801362996, 801368058, 801354860, 801356328, 8...",101417874
6,6,"[0, 272, 2953, 3730, 1621, 4807, 5912, 6116, 3...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1979, 11687, 2167, 12355, 11756, 11652, 1214...","[801348893, 801366426, 801349127, 801367824, 8...",101411256
7,7,"[0, 5837, 7257, 9769, 11573, 6736, 14430, 14432]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[14251, 14089, 12355, 13336, 12943, 12803, 14...","[801367527, 801372060, 801367824, 801363037, 8...",101417467
8,8,"[0, 9769, 14430, 14435]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[14251, 13336, 12803, 3130, 12787, 11979, 104...","[801367527, 801363037, 801370236, 801350278, 8...",101403186
9,9,"[0, 207, 694, 999, 1142, 2107, 2198, 680, 2402...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[12382, 13009, 10937, 13830, 14043, 13211, 12...","[801371813, 801363811, 801364127, 801371092, 8...",101420018


In [55]:
full_data['decoded_user_id'] = full_data.user_id.apply(lambda x: idx2user[x])

final_dict = full_data[['decoded_user_id', 'preds']].to_dict('index')
final_dict = {v['decoded_user_id']: v['preds'] for k, v in final_dict.items()}

import json
with open('../src/app/data/predictions.json', 'w') as f:
    json.dump(final_dict, f, indent=4)

In [48]:
final_dict

{0: {'decoded_user_id': 101352023,
  'preds': [801363037,
   801367596,
   801367854,
   801371327,
   801369803,
   801368748,
   801367799,
   801366911,
   801371341,
   801368439]},
 1: {'decoded_user_id': 101385462,
  'preds': [801368710,
   801366648,
   801368600,
   801356150,
   801351113,
   801348715,
   801364937,
   801371527,
   801364322,
   801362209]},
 2: {'decoded_user_id': 101421897,
  'preds': [801363037,
   801367527,
   801350278,
   801370236,
   801369752,
   801368313,
   801365341,
   801370132,
   801363515,
   801362710]},
 3: {'decoded_user_id': 101354499,
  'preds': [801363037,
   801368178,
   801370014,
   801368600,
   801370992,
   801371145,
   801362957,
   801366089,
   801367847,
   801367146]},
 4: {'decoded_user_id': 101421312,
  'preds': [801367527,
   801370236,
   801350278,
   801368313,
   801365341,
   801363037,
   801363515,
   801362710,
   801370228,
   801363373]},
 5: {'decoded_user_id': 101417874,
  'preds': [801362996,
   801368058

In [56]:
!cp ../data/groups.csv ../src/app/data/.